In [2]:
%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

# TO USE A DATABASE OTHER THAN SQLITE, USE THIS LINE
# Note that this is necessary for parallel execution amongst other things...
# os.environ['SNORKELDB'] = 'postgres:///snorkel-intro'

from snorkel import SnorkelSession
session = SnorkelSession()


from snorkel.models import candidate_subclass

ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])

train_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 0).all()
dev_cands = session.query(ChemicalDisease).filter(ChemicalDisease.split == 1).all()
print(len(train_cands),len(dev_cands))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
8272 888


In [4]:
# from util import load_external_labels

# %time load_external_labels(session, Spouse, annotator_name='gold')

from snorkel.annotations import load_gold_labels

#L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
#L_gold_test = load_gold_labels(session, annotator_name='gold', split=2, zero_one=True)

# L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1)
# L_gold_test = load_gold_labels(session, annotator_name='gold', split=2)
L_gold_dev  = load_gold_labels(session, annotator_name='gold', split=1, zero_one=True)
# gold_labels_dev = [L[0,0] if L[0,0]==1 else -1 for L in L_gold_dev]
gold_labels_dev = [L[0,0] for L in L_gold_dev]


from snorkel.learning.utils import MentionScorer

Instructions for updating:
Use the retry module or similar alternatives.


In [5]:
#gold_labels_dev = [x[0,0] for x in L_gold_dev.todense()]
#for i,L in enumerate(gold_labels_dev):
#    print(i,gold_labels_dev[i])

# gold_labels_dev = []
# for i,L in enumerate(L_gold_dev):
#     gold_labels_dev.append(L[0,0])
    
# gold_labels_test = []
# for i,L in enumerate(L_gold_test):
#     gold_labels_test.append(L[0,0])
    
# print(len(gold_labels_dev),len(gold_labels_test))
# print(gold_labels_dev.count(1),gold_labels_dev.count(-1))
# print(len(gold_labels_dev))

print(gold_labels_dev.count(1),gold_labels_dev.count(0))
print(len(gold_labels_dev))

296 592
888


In [4]:
from gensim.parsing.preprocessing import STOPWORDS
import gensim.matutils as gm

from gensim.models.keyedvectors import KeyedVectors

# Load pretrained model (since intermediate data is not included, the model cannot be refined with additional data)
model = KeyedVectors.load_word2vec_format('../glove_w2v.txt', binary=False)  # C binary format


wordvec_unavailable= set()
def write_to_file(wordvec_unavailable):
    with open("wordvec_unavailable.txt","w") as f:
        for word in wordvec_unavailable:
            f.write(word+"\n")

def preprocess(tokens):
    btw_words = [word for word in tokens if word not in STOPWORDS]
    btw_words = [word for word in btw_words if word.isalpha()]
    return btw_words

def get_word_vectors(btw_words): # returns vector of embeddings of words
    word_vectors= []
    for word in btw_words:
        try:
            word_v = np.array(model[word])
            word_v = word_v.reshape(len(word_v),1)
            #print(word_v.shape)
            word_vectors.append(model[word])
        except:
            wordvec_unavailable.add(word)
    return word_vectors

def get_similarity(word_vectors,target_word): # sent(list of word vecs) to word similarity
    similarity = 0
    target_word_vector = 0
    try:
        target_word_vector = model[target_word]
    except:
        wordvec_unavailable.add(target_word+" t")
        return similarity
    target_word_sparse = gm.any2sparse(target_word_vector,eps=1e-09)
    for wv in word_vectors:
        wv_sparse = gm.any2sparse(wv, eps=1e-09)
        similarity = max(similarity,gm.cossim(wv_sparse,target_word_sparse))
    return similarity


In [6]:
import bz2
from six.moves.cPickle import load

with bz2.BZ2File('data/ctd.pkl.bz2', 'rb') as ctd_f:
    ctd_unspecified, ctd_therapy, ctd_marker = load(ctd_f)

In [20]:
##### Discrete #########

def cand_in_ctd_unspecified(c):
    return 1 if c.get_cids() in ctd_unspecified else 0

def cand_in_ctd_therapy(c):
    return 1 if c.get_cids() in ctd_therapy else 0

def cand_in_ctd_marker(c):
    return 1 if c.get_cids() in ctd_marker else 0


def LF_in_ctd_unspecified(c):
    if(cand_in_ctd_unspecified(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_therapy(c):
    if(cand_in_ctd_therapy(c)==1):
        return (-1,1)
    else:
        return (0,0)

def LF_in_ctd_marker(c):
    if(cand_in_ctd_marker(c)==1):
        return (1,1)
    else:
        return (0,0)

import re
from snorkel.lf_helpers import (
    get_tagged_text,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

# List to parenthetical
def ltp(x):
    return '(' + '|'.join(x) + ')'

def LF_induce(c):
    return (1,1) if re.search(r'{{A}}.{0,20}induc.{0,20}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

causal_past = ['induced', 'caused', 'due']
def LF_d_induced_by_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + '.{0,9}(by|to).{0,50}', 1)==1):
        return (1,1)
    return (0,0)

def LF_d_induced_by_c_tight(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(causal_past) + ' (by|to) ', 1)==1):
        return (1,1)
    return (0,0)

def LF_induce_name(c):
    return (1,1) if 'induc' in c.chemical.get_span().lower() else (0,0)     

causal = ['cause[sd]?', 'induce[sd]?', 'associated with']
def LF_c_cause_d(c):
    return (1,1) if (
        re.search(r'{{A}}.{0,50} ' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
        and not re.search('{{A}}.{0,50}(not|no).{0,20}' + ltp(causal) + '.{0,50}{{B}}', get_tagged_text(c), re.I)
    ) else (0,0)

treat = ['treat', 'effective', 'prevent', 'resistant', 'slow', 'promise', 'therap']
def LF_d_treat_c(c):
    if (rule_regex_search_btw_BA(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d(c):
    if (rule_regex_search_btw_AB(c, '.{0,50}' + ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_treat_d(c):
    if (rule_regex_search_before_B(c, ltp(treat) + '.{0,50}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_treat_d_wide(c):
    if (rule_regex_search_btw_AB(c, '.{0,200}' + ltp(treat) + '.{0,200}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_c_d(c):
    return (1,1) if ('{{A}} {{B}}' in get_tagged_text(c)) else (0,0)

def LF_c_induced_d(c):
    return (1,1) if (
        ('{{A}} {{B}}' in get_tagged_text(c)) and 
        (('-induc' in c[0].get_span().lower()) or ('-assoc' in c[0].get_span().lower()))
        ) else (0,0)

def LF_improve_before_disease(c):
    if(rule_regex_search_before_B(c, 'improv.*', -1)==-1):
        return (-1,1)
    return (0,0)

pat_terms = ['in a patient with ', 'in patients with']
def LF_in_patient_with(c):
    return (-1,1) if re.search(ltp(pat_terms) + '{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

uncertain = ['combin', 'possible', 'unlikely']
def LF_uncertain(c):
    if (rule_regex_search_before_A(c, ltp(uncertain) + '.*', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_induced_other(c):
    if (rule_regex_search_tagged_text(c, '{{A}}.{20,1000}-induced {{B}}', -1)==-1):
        return (-1,1)
    return (0,0)  

def LF_far_c_d(c):
    if (rule_regex_search_btw_AB(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_far_d_c(c):
    if (rule_regex_search_btw_BA(c, '.{100,5000}', -1)==-1):
        return (-1,1)
    return (0,0)

def LF_risk_d(c):
    if (rule_regex_search_before_B(c, 'risk of ', 1)==1):
        return (1,1)
    return (0,0)

def LF_develop_d_following_c(c):
    return (1,1) if re.search(r'develop.{0,25}{{B}}.{0,25}following.{0,25}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

procedure, following = ['inject', 'administrat'], ['following']
def LF_d_following_c(c):
    return (1,1) if re.search('{{B}}.{0,50}' + ltp(following) + '.{0,20}{{A}}.{0,50}' + ltp(procedure), get_tagged_text(c), flags=re.I) else (0,0)

def LF_measure(c):
    return (-1,1) if re.search('measur.{0,75}{{A}}', get_tagged_text(c), flags=re.I) else (0,0)

def LF_level(c):
    return (-1,1) if re.search('{{A}}.{0,25} level', get_tagged_text(c), flags=re.I) else (0,0)

def LF_neg_d(c):
    return (-1,1) if re.search('(none|not|no) .{0,25}{{B}}', get_tagged_text(c), flags=re.I) else (0,0)

WEAK_PHRASES = ['none', 'although', 'was carried out', 'was conducted',
                'seems', 'suggests', 'risk', 'implicated',
               'the aim', 'to (investigate|assess|study)']

WEAK_RGX = r'|'.join(WEAK_PHRASES)

def LF_weak_assertions(c):
    return (-1,1) if re.search(WEAK_RGX, get_tagged_text(c), flags=re.I) else (0,0)


def LF_ctd_marker_c_d(c):
    l,s = LF_c_d(c)
    cl = cand_in_ctd_marker(c)
    return (l*cl,s*cl)

def LF_ctd_marker_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_marker(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)

def LF_ctd_therapy_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_therapy(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_treat(c):
    l,s = LF_c_treat_d_wide(c)
    cl = cand_in_ctd_unspecified(c)
    return (l*cl,s*cl)

def LF_ctd_unspecified_induce(c):
    l1,s1 = LF_c_induced_d(c)
    l2,s2 = LF_d_induced_by_c_tight(c)
    cl = cand_in_ctd_unspecified(c)
    return ((l1 or l2)*cl,max(s1,s2)*cl)


# def LF_ctd_marker_c_d(c):
#     return LF_c_d(c) * cand_in_ctd_marker(c)

# def LF_ctd_marker_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_marker(c)

# def LF_ctd_therapy_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_therapy(c)

# def LF_ctd_unspecified_treat(c):
#     return LF_c_treat_d_wide(c) * cand_in_ctd_unspecified(c)

# def LF_ctd_unspecified_induce(c):
#     return (LF_c_induced_d(c) or LF_d_induced_by_c_tight(c)) * cand_in_ctd_unspecified(c)

def LF_closer_chem(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical closer than @dist/2 in either direction
    sent = c.get_parent()
    closest_other_chem = float('inf')
    for i in range(dis_end, min(len(sent.words), dis_end + dist // 2)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    for i in range(max(0, dis_start - dist // 2), dis_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Chemical' and cid != sent.entity_cids[chem_start]:
            return (-1,1)
    return (0,0)

def LF_closer_dis(c):
    # Get distance between chemical and disease
    chem_start, chem_end = c.chemical.get_word_start(), c.chemical.get_word_end()
    dis_start, dis_end = c.disease.get_word_start(), c.disease.get_word_end()
    if dis_start < chem_start:
        dist = chem_start - dis_end
    else:
        dist = dis_start - chem_end
    # Try to find chemical disease than @dist/8 in either direction
    sent = c.get_parent()
    for i in range(chem_end, min(len(sent.words), chem_end + dist // 8)):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    for i in range(max(0, chem_start - dist // 8), chem_start):
        et, cid = sent.entity_types[i], sent.entity_cids[i]
        if et == 'Disease' and cid != sent.entity_cids[dis_start]:
            return (-1,1)
    return (0,0)


LFs = [LF_c_cause_d,LF_c_d,LF_c_induced_d,LF_c_treat_d,LF_c_treat_d_wide,LF_closer_chem,
    LF_closer_dis,LF_ctd_marker_c_d,LF_ctd_marker_induce,LF_ctd_therapy_treat,
    LF_ctd_unspecified_treat,LF_ctd_unspecified_induce,LF_d_following_c,
    LF_d_induced_by_c,LF_d_induced_by_c_tight,LF_d_treat_c,LF_develop_d_following_c,
    LF_far_c_d,LF_far_d_c,LF_improve_before_disease,LF_in_ctd_therapy,
    LF_in_ctd_marker,LF_in_patient_with,LF_induce,LF_induce_name,LF_induced_other,
    LF_level,LF_measure,LF_neg_d,LF_risk_d,LF_treat_d,LF_uncertain,LF_weak_assertions
]

LF_l = [
    1,1,1,-1,-1,-1,
    -1,1,1,-1,
    -1,1,1,
    1,1,-1,1,
    -1,-1,-1,-1,
    1,-1,1,1,-1,
    -1,-1,-1,1,-1,-1,-1
]
print(len(LFs),len(LF_l))

33 33


In [21]:
''' output:

    [[[L_x1],[S_x1]],
     [[L_x2],[S_x2]],
     ......
     ......
    ]

'''
def get_L_S_Tensor(cands): 
    
    L_S = []
    for i,ci in enumerate(cands):
        L_S_ci=[]
        L=[]
        S=[]
        P_ik = []
        for LF in LFs:
            #print LF.__name__
            l,s = LF(ci)
            L.append(l)
            S.append((s+1)/2)  #to scale scores in [0,1] 
        L_S_ci.append(L)
        L_S_ci.append(S)
        L_S.append(L_S_ci) 
        if(i%500==0 and i!=0):
            print(str(i)+'data points labelled in',(time.time() - start_time)/60,'mins')
    return L_S


In [16]:
# import matplotlib.pyplot as plt
import time
import numpy as np
start_time = time.time()

lt = time.localtime()

print("started at: {}-{}-{}, {}:{}:{}".format(lt.tm_mday,lt.tm_mon,lt.tm_year,lt.tm_hour,lt.tm_min,lt.tm_sec))

dev_L_S = get_L_S_Tensor(dev_cands)
np.save("dev_L_S_discrete",np.array(dev_L_S))

train_L_S = get_L_S_Tensor(train_cands)
np.save("train_L_S_discrete",np.array(train_L_S))

print("--- %s seconds ---" % (time.time() - start_time))

# test_L_S = get_L_S_Tensor(test_cands)
# pkl.dump(test_L_S,open("test_L_S.p","wb"))

started at: 7-5-2018, 13:38:29
500data points labelled in 0.024617326259613038 mins
500data points labelled in 0.06944836378097534 mins
1000data points labelled in 0.10239864190419515 mins
1500data points labelled in 0.12567600806554158 mins
2000data points labelled in 0.1551079233487447 mins
2500data points labelled in 0.1808343489964803 mins
3000data points labelled in 0.2076396902402242 mins
3500data points labelled in 0.23080986340840656 mins
4000data points labelled in 0.2608593622843424 mins
4500data points labelled in 0.2797205646832784 mins
5000data points labelled in 0.30112220843633014 mins
5500data points labelled in 0.32616100311279295 mins
6000data points labelled in 0.3475878000259399 mins
6500data points labelled in 0.3676290432612101 mins
7000data points labelled in 0.39147427479426067 mins
7500data points labelled in 0.4106962045033773 mins
8000data points labelled in 0.4307020902633667 mins
--- 26.474006175994873 seconds ---


In [17]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def draw2DArray(a):
    fig = plt.figure(figsize=(6, 3.2))
    ax = fig.add_subplot(111)
    ax.set_title('colorMap')
    plt.imshow(np.array(a))
    ax.set_aspect('equal')
    cax = fig.add_axes([0.12, 0.1, 0.78, 0.8])
    cax.get_xaxis().set_visible(False)
    cax.get_yaxis().set_visible(False)
    cax.patch.set_alpha(0)
    cax.set_frame_on(False)
    plt.colorbar(orientation='vertical')
    plt.show()
    
      
def report2dict(cr):
    # Parse rows
    tmp = list()
    for row in cr.split("\n"):
        parsed_row = [x for x in row.split("  ") if len(x) > 0]
        if len(parsed_row) > 0:
            tmp.append(parsed_row)
    
    # Store in dictionary
    measures = tmp[0]

    D_class_data = defaultdict(dict)
    for row in tmp[1:]:
        class_label = row[0]
        for j, m in enumerate(measures):
            D_class_data[class_label][m.strip()] = float(row[j + 1].strip())
    return pd.DataFrame(D_class_data).T

def predictAndPrint(pl):
    print("acc",accuracy_score(gold_labels_dev,pl))
#     print(precision_recall_fscore_support(true_labels,pl,average='macro'))
    print(confusion_matrix(gold_labels_dev,pl))
    draw2DArray(confusion_matrix(gold_labels_dev,pl))
    return report2dict(classification_report(gold_labels_dev, pl))# target_names=class_names))
    
  

In [22]:
import numpy as np
dev_L_S = np.load("dev_L_S_discrete.npy")
train_L_S = np.load("train_L_S_discrete.npy")
print(dev_L_S.shape,train_L_S.shape)

(888, 2, 33) (8272, 2, 33)


In [23]:
#call this only once for a kernel startup
from __future__ import absolute_import, division, print_function

import tensorflow as tf
# BATCH_SIZE = 32

In [24]:
NoOfLFs= len(LFs)
NoOfClasses = 2
print(len(LFs),len(LF_l))

33 33


In [25]:
## class wise reproduced using map over y
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

    thetas = tf.get_variable('thetas', [1,NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(1,0.1,seed),\
                             dtype=tf.float64)
    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(5):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 33), dtype=float64)
k Tensor("Const_1:0", shape=(33,), dtype=float64)
<tf.Variable 'alphas:0' shape=(33,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
nls Tensor("mul:0", shape=(?, 33), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 33) dtype=float64_ref>
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(33,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo

In [25]:
## class wise reproduced using map over y
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

    thetas = tf.get_variable('thetas', [1,NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(1,0.1,seed),\
                             dtype=tf.float64)
    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(5):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl,l = sess.run([alphas,thetas,marginals,predict,normloss])
#             print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

#             unique, counts = np.unique(pl, return_counts=True)
#             print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 33), dtype=float64)
k Tensor("Const_1:0", shape=(33,), dtype=float64)
<tf.Variable 'alphas:0' shape=(33,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 33), dtype=float64)
l Tensor("unstack:0", shape=(?, 33), dtype=float64)
nls Tensor("mul:0", shape=(?, 33), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 33) dtype=float64_ref>
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 33), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(33,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo

In [28]:
## varying z weight 

def train(z_weight):
    BATCH_SIZE = 1
    tf.reset_default_graph()

    seed = 12
    with tf.Graph().as_default():

        train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
        dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

        labels = tf.convert_to_tensor(gold_labels_dev)
        iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                               train_dataset.output_shapes)
        next_element = iterator.get_next()

        train_init_op = iterator.make_initializer(train_dataset)
        dev_init_op = iterator.make_initializer(dev_dataset)
        
        zw = tf.convert_to_tensor(z_weight,dtype=tf.float64)

        next_element = iterator.get_next()
#         print("next_element",next_element)

        alphas = tf.get_variable('alphas', [NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                                 dtype=tf.float64)

        thetas = tf.get_variable('thetas', [1,NoOfLFs],\
                                 initializer=tf.truncated_normal_initializer(1,0.1,seed),\
                                 dtype=tf.float64)

        k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
#         print("k",k)
#         print(alphas.graph)
#         print(thetas.graph)
        l,s =  tf.unstack(next_element,axis=1)
#         print(alphas)
#         print(s)
#         print("l",l)
#         print(s.graph)

        s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)



        ls_ = tf.multiply(l,s_)

        nls_ = tf.multiply(l,s_)*-1

        pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
#         print("nls",nls_)
#         print("thetas",thetas)

    #     lst = tf.matmul(ls_,thetas)
    #     print("lst",lst)
        t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
#         print("pout",pout)

#         print("t_pout",t_pout)

        t_k =  k*tf.squeeze(thetas)
#         print("t_k",t_k)

        zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))

        logz = tf.log(tf.reduce_sum(zy,axis=0))
#         print("zy",zy)
#         print("logz",logz)

        lsp = tf.reduce_logsumexp(t_pout)
#         print("lsp",lsp)

    #     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

        normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - zw*logz))


#         print("normloss",normloss)
        marginals = tf.nn.softmax(t_pout,axis=0)

#         print("marginals",marginals)
        predict = tf.argmax(marginals,axis=0)
#         print("predict",predict)

    #     pre = tf.metrics.precision(labels,predict)
    #     rec = tf.metrics.recall(labels,predict)
    #     print("loss",loss)
    #     print("nls_",nls_)

    #     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
    #     starter_learning_rate = 1.0
    #     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
    #                                            10, 0.96, staircase=True)
    #     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


    #     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
    #     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    #     reg_constant = 5.0  # Choose an appropriate one.
    #     totalloss = normloss + reg_constant * sum(reg_losses)
        train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
    #     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

    #     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

        init_g = tf.global_variables_initializer()
        init_l = tf.local_variables_initializer()
        with tf.Session() as sess:
            sess.run(init_g)
            sess.run(init_l)

            # Initialize an iterator over the training dataset.
            for it in range(1):
                sess.run(train_init_op)
                tl = 0
                try:
                    while True:
                        _,ls = sess.run([train_step,normloss])
                        tl = tl + ls
                except tf.errors.OutOfRangeError:
                    pass
                print(it,"loss",tl)

#                 sess.run(dev_init_op)
#                 a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#                 print(a)
#                 print(t)


    #         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


#                 unique, counts = np.unique(pl, return_counts=True)
#                 print(dict(zip(unique, counts)))
#                 print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
#                 print()

            # Initialize an iterator over the validation dataset.
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)

            MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())


            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))

            print("acc",accuracy_score(gold_labels_dev,pl))

            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
            cf = confusion_matrix(gold_labels_dev,pl)
            print(cf)
            print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      

for w in np.linspace(0,1,10):
    print()
    print("z-weight:",w)
    train(w)


z-weight: 0.0
0 loss -227408.91347570377
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233  0.18357064 0.26661405
 0.18238184 0.27858159 0.24081247 0.24223612 0.05334134 0.11993454
 0.25962482 0.03641602 0.23537624 0.14143537 0.19621468 0.36301113
 0.07373343 0.37747756 0.26808572 0.26850778 0.25747627 0.19034372
 0.02199156 0.03837747 0.16215054]
[[ 14.67744856  25.90583748  22.69587475  12.78045065  14.39716461
    0.44035997  -3.2445657   25.08846134  24.96396903  13.96286426
   11.82652493  22.46522151   1.79888645  16.7243985   11.77938823
   -3.43412061   2.63100655   1.29551022  -6.77434535  -0.59566388
  -10.15505727  61.98589432   1.58045045  12.01429343  28.43790004
    3.19948727   1.48865339  -1.91094183  -4.65161603   5.80171757
   -2.764758     1.47742811 -15.12888705]]
Scores (Un-adjusted)
Pos. class accuracy: 1.0
Neg. class accuracy: 0.314
Precision            0.422
Recall               1.0
F1                

0 loss 96689.88058612194
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233  0.18357064 0.26661405
 0.18238184 0.27858159 0.24081247 0.24223612 0.05334134 0.11993454
 0.25962482 0.03641602 0.23537624 0.14143537 0.19621468 0.36301113
 0.07373343 0.37747756 0.26808572 0.26850778 0.25747627 0.19034372
 0.02199156 0.03837747 0.16215054]
[[-0.82561982 -0.36723542 -0.55485411  1.03417808  1.27583029  0.71717905
   0.77682786 -0.36610815 -0.4850003   1.0696494   1.17258026 -0.55315662
  -1.07879907 -0.86485836 -0.96553447  0.97462023 -1.0725238   0.90759641
   0.73696937  1.05145869  0.54532653 28.29318095  1.07189892 -0.90287243
  -0.12768098  0.91690854  1.0383057   1.05647057  1.02109501 -0.99756409
   0.97302553  0.89269963  0.79117205]]
Scores (Un-adjusted)
Pos. class accuracy: 0.993
Neg. class accuracy: 0.598
Precision            0.553
Recall               0.993
F1                   0.71
----------------------------------------

In [42]:
## init thetas with snorkel thetas and train
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

#     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
#                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
#                              dtype=tf.float64)
    
    thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                            initializer=tf.constant_initializer(np.array([[0.07472098,\
                            0.07514459,  0.11910277,0.11186369,0.07306518,0.69216714,\
                            0.07467749,0.16012659, 0.13682546,0.08183363]])),\
                            dtype=tf.float64)
    print("thetas",thetas)
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(5):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
k Tensor("Const_1:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo

In [45]:
## init thetas with old network thetas and train
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

#     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
#                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
#                              dtype=tf.float64)
    
    thetas = tf.get_variable('thetas',[1,NoOfLFs],\
                            initializer=tf.constant_initializer(np.array([[1.0,1.0,1.0,\
                             1.0,1.0,1.02750979,1.0,1.0218145,1.0,1.0]])),\
                    dtype=tf.float64)
                             
    print("thetas",thetas)
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(5):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
k Tensor("Const_1:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo

In [29]:
## Objective value on snorkel thetas
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

#     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
#                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
#                              dtype=tf.float64)
    
    thetas = tf.convert_to_tensor(np.array([[ 0.07472098,  0.07514459,  0.11910277,\
            0.11186369,0.07306518,0.69216714,0.07467749,0.16012659, 0.13682546,0.08183363]]))

    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
#     train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(1):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
#                     _,ls = sess.run([train_step,normloss])
                    ls = sess.run(normloss) # to calculate loss on fixed thetas
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
k Tensor("Const_2:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
thetas Tensor("Const_1:0", shape=(1, 10), dtype=float64)
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 1535

In [34]:
## Objective value on thetas from old network

BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

#     thetas = tf.get_variable('thetas', [1,NoOfLFs],\
#                              initializer=tf.truncated_normal_initializer(1,0.1,seed),\
#                              dtype=tf.float64)
    
    thetas = tf.convert_to_tensor(np.array([[1.0,1.0,1.0,1.0,1.0,1.02750979,\
                             1.0,1.0218145,1.0,1.0]]))
    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
#     train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(1):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
#                     _,ls = sess.run([train_step,normloss])
                    ls = sess.run(normloss) # to calculate loss on fixed thetas
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
#             print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
        MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
k Tensor("Const_2:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
thetas Tensor("Const_1:0", shape=(1, 10), dtype=float64)
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 loss 1795

In [88]:
## same network that didn't train
BATCH_SIZE = 1
tf.reset_default_graph()

seed = 12
with tf.Graph().as_default():
    
    train_dataset = tf.data.Dataset.from_tensor_slices(train_L_S).batch(BATCH_SIZE)
    dev_dataset = tf.data.Dataset.from_tensor_slices(dev_L_S).batch(dev_L_S.shape[0])

    labels = tf.convert_to_tensor(gold_labels_dev)
    iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
    next_element = iterator.get_next()

    train_init_op = iterator.make_initializer(train_dataset)
    dev_init_op = iterator.make_initializer(dev_dataset)
   
    next_element = iterator.get_next()
    print("next_element",next_element)

    alphas = tf.get_variable('alphas', [NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(0.2,0.1,seed),\
                             dtype=tf.float64)

    thetas = tf.get_variable('thetas', [1,NoOfLFs],\
                             initializer=tf.truncated_normal_initializer(1,0.1,seed),\
                             dtype=tf.float64)
    
    k = tf.convert_to_tensor(LF_l, dtype=tf.float64)
    print("k",k)
    print(alphas.graph)
    print(thetas.graph)
    l,s =  tf.unstack(next_element,axis=1)
    print(alphas)
    print(s)
    print("l",l)
    print(s.graph)

    s_ = tf.map_fn(lambda x : tf.maximum(tf.subtract(x,alphas), 0), s)

            
        
    ls_ = tf.multiply(l,s_)

    nls_ = tf.multiply(l,s_)*-1
    
    pout = tf.map_fn(lambda x: l*x,np.array([-1,1],dtype=np.float64))
    print("nls",nls_)
    print("thetas",thetas)
    
#     lst = tf.matmul(ls_,thetas)
#     print("lst",lst)
    t_pout = tf.map_fn(lambda x: tf.matmul(x,thetas,transpose_b=True),pout)
    print("pout",pout)
    
    print("t_pout",t_pout)

    t_k =  k*tf.squeeze(thetas)
    print("t_k",t_k)
    
    zy = tf.map_fn(lambda y: tf.reduce_prod(1+tf.exp(t_k*y),axis=0),np.array([-1,1],dtype=np.float64))
    
    logz = tf.log(tf.reduce_sum(zy,axis=0))
    print("zy",zy)
    print("logz",logz)
    
    lsp = tf.reduce_logsumexp(t_pout)
    print("lsp",lsp)

#     normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0)) - logz)  # add z

    normloss = tf.negative(tf.reduce_sum(tf.reduce_logsumexp(t_pout,axis=0) - logz))


    print("normloss",normloss)
    marginals = tf.nn.softmax(t_pout,axis=0)
      
    print("marginals",marginals)
    predict = tf.argmax(marginals,axis=0)
    print("predict",predict)
    
#     pre = tf.metrics.precision(labels,predict)
#     rec = tf.metrics.recall(labels,predict)
#     print("loss",loss)
#     print("nls_",nls_)

#     global_step = tf.Variable(0, trainable=False,dtype=tf.float64)
#     starter_learning_rate = 1.0
#     learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
#                                            10, 0.96, staircase=True)
#     train_step = tf.train.AdamOptimizer(learning_rate).minimize(normloss, global_step=global_step) 


#     train_step = tf.train.AdamOptimizer(0.001).minimize(normloss)
#     reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
#     reg_constant = 5.0  # Choose an appropriate one.
#     totalloss = normloss + reg_constant * sum(reg_losses)
    train_step = tf.train.AdamOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.AdagradOptimizer(0.01).minimize(normloss) 
#     train_step = tf.train.MomentumOptimizer(0.01,0.2).minimize(normloss) 

#     train_step = tf.train.GradientDescentOptimizer(0.1).minimize(normloss)

    init_g = tf.global_variables_initializer()
    init_l = tf.local_variables_initializer()
    with tf.Session() as sess:
        sess.run(init_g)
        sess.run(init_l)
        
        # Initialize an iterator over the training dataset.
        for it in range(30):
            sess.run(train_init_op)
            tl = 0
            try:
                while True:
                    _,ls = sess.run([train_step,normloss])
                    tl = tl + ls
            except tf.errors.OutOfRangeError:
                pass
            print(it,"loss",tl)
            
            sess.run(dev_init_op)
            a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
            print(a)
            print(t)
          
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
            

            unique, counts = np.unique(pl, return_counts=True)
            print(dict(zip(unique, counts)))
            print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))
            print()
            
        # Initialize an iterator over the validation dataset.
        sess.run(dev_init_op)
        a,t,m,pl = sess.run([alphas,thetas,marginals,predict])
        print(a)
        print(t)
                      
#         MentionScorer(dev_cands, L_gold_dev).score(m[1::].flatten())
    
       
        unique, counts = np.unique(pl, return_counts=True)
        print(dict(zip(unique, counts)))
       
        print("acc",accuracy_score(gold_labels_dev,pl))
  
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl)))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="macro"))
        cf = confusion_matrix(gold_labels_dev,pl)
        print(cf)
        print("prec: tp/(tp+fp)",cf[1][1]/(cf[1][1]+cf[0][1]),"recall: tp/(tp+fn)",cf[1][1]/(cf[1][1]+cf[1][0]))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(pl),average="binary"))

    
      
            


next_element Tensor("IteratorGetNext_1:0", shape=(?, 2, 10), dtype=float64)
k Tensor("Const_1:0", shape=(10,), dtype=float64)
<tf.Variable 'alphas:0' shape=(10,) dtype=float64_ref>
Tensor("unstack:1", shape=(?, 10), dtype=float64)
l Tensor("unstack:0", shape=(?, 10), dtype=float64)
nls Tensor("mul:0", shape=(?, 10), dtype=float64)
thetas <tf.Variable 'thetas:0' shape=(1, 10) dtype=float64_ref>
pout Tensor("map_1/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 10), dtype=float64)
t_pout Tensor("map_2/TensorArrayStack/TensorArrayGatherV3:0", shape=(2, ?, 1), dtype=float64)
t_k Tensor("mul_1:0", shape=(10,), dtype=float64)
zy Tensor("map_3/TensorArrayStack/TensorArrayGatherV3:0", shape=(2,), dtype=float64)
logz Tensor("Log:0", shape=(), dtype=float64)
lsp Tensor("ReduceLogSumExp/add:0", shape=(), dtype=float64)
normloss Tensor("Neg:0", shape=(), dtype=float64)
marginals Tensor("transpose_1:0", shape=(2, ?, 1), dtype=float64)
predict Tensor("ArgMax:0", shape=(?, 1), dtype=int64)
0 lo

20 loss 145734.55921424946
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.61788499  0.58264704  0.66905544  0.66786296  0.48311329  5.59198573
   0.61793172 -0.2656293  -0.36412623 -0.67924585]]
{0: 2362, 1: 452}
(0.07079646017699115, 0.1693121693121693, 0.09984399375975038, None)

21 loss 145734.55921424946
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.61788499  0.58264704  0.66905544  0.66786296  0.48311329  5.59198573
   0.61793172 -0.2656293  -0.36412623 -0.67924585]]
{0: 2362, 1: 452}
(0.07079646017699115, 0.1693121693121693, 0.09984399375975038, None)

22 loss 145734.55921424946
[0.31754463 0.01421453 0.20955965 0.20062431 0.30637787 0.24133855
 0.369206   0.23120785 0.34633471 0.2647233 ]
[[ 0.61788499  0.58264704  0.66905544  0.66786296  0.48311329  5.59198573
   0.61793172 -0.2656293  -0.36412623 -0.67924585]]
{0: 2362, 1: 452}
(0.07079

In [224]:
#snorkel
a =np.array([ 0.07472098,  0.07514459,  0.11910277,  0.11186369,  0.07306518,
        0.69216714,  0.07467749,  0.16012659,  0.13682546,  0.08183363])

temp = a.flatten().argsort()
ranks = np.empty_like(temp)
ranks[temp] = np.arange(len(a))
print(temp)

[4 6 0 1 9 3 2 8 7 5]


In [27]:
a = np.array([-0.27082211, -0.01928787, -0.14063245,  0.37856253,  0.43681819, -0.15844807,
   0.13280198, -0.01935702, -0.10775934,  0.34390113,  0.39762823, -0.14286955,
  -0.39588527, -0.33699178, -0.37821404,  0.38378715, -0.39537146,  0.11504936,
   0.21906794,  0.39699417, -0.27113816,  8.13838832,  0.39548336, -0.31328908,
   0.25503373,  0.28019293,  0.39734506,  0.39700564,  0.37866251, -0.39156514,
   0.38002959,  0.21917987,  0.13605525])

temp = a.flatten().argsort()
ranks = np.empty_like(temp)
ranks[temp] = np.arange(len(a))
print(temp)

[12 16 29 14 13 23 20  0  5 11  2  8  7  1 17  6 32 18 31 24 25  9  3 28
 30 15 22 19 27 26 10  4 21]


In [31]:
# rerun old network to get thetas
#stochastic + weighted cross entropy logits func + remove min(theta,0) in loss -- Marked
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

def train_NN():
    print()
    result_dir = "./"
    config = projector.ProjectorConfig()
    tf.logging.set_verbosity(tf.logging.INFO)
    summary_writer = tf.summary.FileWriter(result_dir)

    tf.reset_default_graph()

    dim = 2 #(labels,scores)

    _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))

    alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
                            dtype=tf.float64)

    thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(1),
                            dtype=tf.float64)

    l,s = tf.unstack(_x)

    prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        

    mul_L_S = tf.multiply(l,prelu_out_s)

    phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

    phi_n1 = tf.reduce_sum(tf.multiply(tf.negative(mul_L_S),thetas))

    phi_out = tf.stack([phi_n1,phi_p1])
    
    predict = tf.argmax(tf.nn.softmax(phi_out))

    loss = tf.negative(tf.reduce_logsumexp(phi_out))

    train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss) 


    check_op = tf.add_check_numerics_ops()

    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init)

    for i in range(5):
        c = 0
        te_prev=1
        total_te = 0
        for L_S_i in train_L_S:

            a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i})
            total_te+=te_curr

            if(abs(te_curr-te_prev)<1e-200):
                break

            if(c%4000==0):
                pl = []
                for L_S_i in dev_L_S:
                    a,t,de_curr,p = sess.run([alphas,thetas,loss,predict],feed_dict={_x:L_S_i})
                    pl.append(p)
                predicted_labels = pl
                print()
                print(total_te/4000)
                total_te=0
#                 print(a)
                print(t)
#                 print()
                print(predicted_labels.count(-1),predicted_labels.count(1))
                print(c," d ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
            c+=1
            te_prev = te_curr
        pl = []
        for L_S_i in dev_L_S:
            p = sess.run(predict,feed_dict={_x:L_S_i})
            pl.append(p)
        predicted_labels = pl
        print(i,total_te)
        print(predicted_labels.count(-1),predicted_labels.count(1))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
        print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='binary'))
    
train_NN()



-0.00024597518522208474
[1.         1.         1.         1.         1.         1.00531229
 1.         1.         1.         1.        ]
0 292
0  d  (0.6593199026647693, 0.7364867724867725, 0.6874226231133167, None)
0 -0.9839007408883389
0 292
(0.6593199026647693, 0.7364867724867725, 0.6874226231133167, None)
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

-0.0002477972163026618
[1.         1.         1.         1.         1.         1.01071759
 1.         1.00531229 1.         1.        ]
0 292
0  d  (0.6593199026647693, 0.7364867724867725, 0.6874226231133167, None)
1 -0.9911888652106471
0 292
(0.6593199026647693, 0.7364867724867725, 0.6874226231133167, None)
(0.3527397260273973, 0.544973544973545, 0.4282744282744283, None)

-0.00024967684220414605
[1.         1.         1.         1.         1.         1.01621772
 1.         1.01071759 1.         1.        ]
0 292
0  d  (0.6593199026647693, 0.7364867724867725, 0.6874226231133167, None)
2 -0.9987073688165843
0 292

In [85]:
# #stochastic + weighted cross entropy logits func + remove min(theta,0) in loss -- Marked
# from __future__ import absolute_import
# from __future__ import division
# from __future__ import print_function
# import tensorflow as tf
# from tensorflow.contrib.tensorboard.plugins import projector

# def train_NN():
#     print()
#     result_dir = "./"
#     config = projector.ProjectorConfig()
#     tf.logging.set_verbosity(tf.logging.INFO)
#     summary_writer = tf.summary.FileWriter(result_dir)

#     tf.reset_default_graph()

#     dim = 2 #(labels,scores)

#     _x = tf.placeholder(tf.float64,shape=(dim,len(LFs)))

#     alphas = tf.get_variable('alpha', _x.get_shape()[-1],initializer=tf.constant_initializer(0.2),
#                             dtype=tf.float64)

#     thetas = tf.get_variable('theta', _x.get_shape()[-1],initializer=tf.constant_initializer(1),
#                             dtype=tf.float64)

#     l,s = tf.unstack(_x)

#     prelu_out_s = tf.maximum(tf.subtract(s,alphas), tf.zeros(shape=(len(LFs)),dtype=tf.float64))        

#     mul_L_S = tf.multiply(l,prelu_out_s)

#     phi_p1 = tf.reduce_sum(tf.multiply(mul_L_S,thetas))

#     phi_n1 = tf.reduce_sum(tf.multiply(tf.negative(mul_L_S),thetas))

#     phi_out = tf.stack([phi_n1,phi_p1])
    
#     predict = tf.argmax(tf.nn.softmax(phi_out))

#     loss = tf.negative(tf.reduce_logsumexp(phi_out))

#     train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss) 


#     check_op = tf.add_check_numerics_ops()

#     sess = tf.Session()
#     init = tf.global_variables_initializer()
#     sess.run(init)

#     for i in range(1):
#         c = 0
#         te_prev=1
#         total_te = 0
#         for L_S_i in train_L_S:

#             a,t,te_curr,_ = sess.run([alphas,thetas,loss,train_step],feed_dict={_x:L_S_i})
#             total_te+=te_curr

#             if(abs(te_curr-te_prev)<1e-200):
#                 break

#             if(c%4000==0):
#                 pl = []
#                 for L_S_i in dev_L_S:
#                     a,t,de_curr,p = sess.run([alphas,thetas,loss,predict],feed_dict={_x:L_S_i})
#                     pl.append(p)
#                 predicted_labels = [-1 if x==0 else x for x in pl]
#                 print()
#                 print(total_te/4000)
#                 total_te=0
# #                 print(a)
# #                 print(t)
# #                 print()
#                 print(predicted_labels.count(-1),predicted_labels.count(1))
#                 print(c," d ",precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
#             c+=1
#             te_prev = te_curr
#         pl = []
#         for L_S_i in dev_L_S:
#             p = sess.run(predict,feed_dict={_x:L_S_i})
#             pl.append(p)
#         predicted_labels = [-1 if x==0 else x for x in pl]
#         print(i,total_te)
#         print(predicted_labels.count(-1),predicted_labels.count(1))
#         print(precision_recall_fscore_support(np.array(gold_labels_dev),np.array(predicted_labels),average='macro'))
    
# train_NN()



-0.000226377142841
2251 545
0  d  (0.58865213829531426, 0.71341836734693875, 0.60408179957052133, None)

-1.94518369934e+28
2232 564
4000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-5.04415736866e+58
2232 564
8000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-1.33431810995e+89
2232 564
12000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-3.67295517678e+119
2232 564
16000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)

-9.52453175821e+149
2232 564
20000  d  (0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)
0 -1.71979948062e+170
2232 564
(0.57825249752154351, 0.6933045525902668, 0.58885935866155448, None)
